# Fake News Classifier Using Bidirectional LSTM

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
###Drop Nan Values
df=df.dropna()

In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:

## Get the Dependent features
y=df['label']

In [ ]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.13.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
### Vocabulary size
voc_size=5000

Onehot Representation

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4203, 3931, 842, 2114, 29, 3556, 3764, 4154, 691, 3406],
 [1257, 3371, 908, 4677, 942, 2696, 2210],
 [2254, 3714, 4708, 356],
 [1269, 3067, 399, 3296, 4394, 4294],
 [396, 942, 1825, 4999, 194, 1696, 942, 1242, 3880, 2864],
 [4289,
  4550,
  1057,
  2931,
  4186,
  1126,
  1031,
  2801,
  4263,
  3694,
  1766,
  1766,
  1597,
  1097,
  2210],
 [4037, 3078, 2425, 425, 1261, 2261, 3016, 3328, 1891, 2617, 655],
 [3557, 925, 105, 4700, 4352, 726, 1126, 124, 1891, 2617, 655],
 [2451, 3424, 4886, 4952, 139, 2888, 1639, 165, 1126, 3372],
 [2720, 4225, 3472, 1716, 2966, 1353, 3828, 444],
 [866, 1077, 564, 3173, 3991, 4290, 2772, 229, 1604, 110, 2901],
 [3296, 448, 29, 2888, 1126, 4352],
 [4316, 4032, 296, 2539, 2299, 3479, 1961, 1012, 1463],
 [4535, 4026, 298, 2364, 4504, 4293, 1852, 1891, 2617, 655],
 [383, 2670, 495, 558, 4473, 1891, 2617, 655],
 [2534, 4298, 755, 555, 4688, 2155, 3564, 4510, 3369, 3118],
 [3185, 1449, 3371],
 [2513, 665, 3486, 468, 1126, 2271, 3962, 2210],
 [3052, 2203, 90

Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4154  691 3406]
 [   0    0    0 ...  942 2696 2210]
 [   0    0    0 ... 3714 4708  356]
 ...
 [   0    0    0 ... 1891 2617  655]
 [   0    0    0 ... 2072 2129 3167]
 [   0    0    0 ... 2068  668  933]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4203,
       3931,  842, 2114,   29, 3556, 3764, 4154,  691, 3406], dtype=int32)

In [ ]:
## Creating model using simple lstm layer
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(LSTM(100))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# print(model.summary())

In [ ]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 23s 82ms/step - loss: 0.2877 - accuracy: 0.8652 - val_loss: 0.2059 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 16s 81ms/step - loss: 0.1395 - accuracy: 0.9466 - val_loss: 0.2165 - val_accuracy: 0.9133
Epoch 3/10
192/192 [==============================] - 15s 76ms/step - loss: 0.0971 - accuracy: 0.9644 - val_loss: 0.2541 - val_accuracy: 0.9110
Epoch 4/10
192/192 [==============================] - 14s 73ms/step - loss: 0.0763 - accuracy: 0.9719 - val_loss: 0.2532 - val_accuracy: 0.9085
Epoch 5/10
192/192 [==============================] - 15s 77ms/step - loss: 0.0509 - accuracy: 0.9833 - val_loss: 0.3570 - val_accuracy: 0.9075
Epoch 6/10
192/192 [==============================] - 16s 85ms/step - loss: 0.0370 - accuracy: 0.9873 - val_loss: 0.3523 - val_accuracy: 0.9064
Epoch 7/10
192/192 [==============================] - 15s 78ms/step - loss: 0.0242 - accuracy: 0.9927 - val_loss: 0.4799 - val_accuracy:

Performance Metrics And Accuracy

In [ ]:
y_pred1 = model1.predict(X_test)

189/189 [==============================] - 2s 10ms/step


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# import numpy as np

# #example explaining what is happening in y_pred

# # Create a multi-dimensional array
# y_pred1 = np.array([[1.234, 2.345], [3.456, 4.567]])

# # Flatten the array
# y_pred1 = y_pred1.flatten()

# # Print the flattened array with 2 decimal places
# print(y_pred1.round(2))

[1.23 2.35 3.46 4.57]


In [ ]:
y_pred1 = y_pred1.flatten()
print(y_pred1.round(2))

[1. 0. 0. ... 0. 1. 1.]


In [ ]:
# extract the predicted class labels
y_final_pred = np.where(y_pred1 > 0.5, 1, 0)
# if the condition is true for an element, it assigns 1 to the corresponding position in y_final_pred. If the condition is false, it assigns 0.
print(y_final_pred)

[1 0 0 ... 0 1 1]


In [ ]:
confusion_matrix(y_test,y_final_pred)

array([[3088,  331],
       [ 221, 2395]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_final_pred)

0.9085335542667772

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_final_pred))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

